In [1]:
import pandas as pd
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
# from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer


In [2]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dayal\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [39]:
df = pd.read_csv('Cleaned_dataset/SarTanSeJuda.csv')

output_csv = r'preprocessed_dataset/SarTanSeJuda.csv'

In [40]:
df.head()

,Unnamed: 0,user_id,username,id,date,Tweet,lang,replyCount,followersCount
0,0,1564135521760096258,SaffronFire,1570051004053491713,2022-09-14 14:05:04+00:00,Sadhus getting keeled In the name of Fake Bacc...,en,0,597
1,1,306357166,SimplyyJKS,1570040889472618497,2022-09-14 13:24:52+00:00,Need this much of confidence 😆 \n\n#LoveJihad ...,en,0,32
2,2,1133093590740025344,rummy_chauhan,1570040767833571333,2022-09-14 13:24:23+00:00,मेरा देश बदल रहा है !💪🏻. 🙏🏻good job &amp;आभ...,hi,0,81
3,3,839718471936126977,HimanshuShrangi,1570033540196339717,2022-09-14 12:55:40+00:00,#Rap_Song \n#SarTanSeJuda \n#SaveIndia \n@HESG...,qme,0,39
4,4,914225518626586624,NirmalKumarTha8,1570031461696438273,2022-09-14 12:47:24+00:00,#SarTanSeJuda se bachao mere desh ko.. https:/...,hi,0,8


In [41]:
df.shape

(6925, 9)

In [42]:
df_eng = df[df.lang == 'en']
df_eng.head()

,Unnamed: 0,user_id,username,id,date,Tweet,lang,replyCount,followersCount
0,0,1564135521760096258,SaffronFire,1570051004053491713,2022-09-14 14:05:04+00:00,Sadhus getting keeled In the name of Fake Bacc...,en,0,597
1,1,306357166,SimplyyJKS,1570040889472618497,2022-09-14 13:24:52+00:00,Need this much of confidence 😆 \n\n#LoveJihad ...,en,0,32
5,5,1393917252303417348,soldierofgod108,1570031351554002946,2022-09-14 12:46:58+00:00,India now chants Bharat Mata ki Jai / Vande Ma...,en,0,76
6,6,1535688169231831040,RanaJi19861986,1570028420788293640,2022-09-14 12:35:19+00:00,@HinduBoy09 Tight slap for #SarTanSeJuda sloga...,en,0,0
7,7,1566728648484331524,AaliyaFatima94,1570025110635008002,2022-09-14 12:22:10+00:00,What is going on?? avoid such kinda of hate......,en,1,20


In [43]:
df.shape, df_eng.shape

((6925, 9), (4654, 9))

In [44]:
df_eng['user_id'].nunique(), df['user_id'].nunique()

(2315, 3424)

In [45]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

## Defining set containing all stopwords in english.
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [46]:
def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    hashtagPattern    = '#[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' URL',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' ', tweet)   
        
        tweet = re.sub(hashtagPattern, ' ', tweet)
        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            #if word not in stopwordlist:
            if len(word)>1:
                # Lemmatizing the word.
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [47]:
import time
text = list(df_eng['Tweet'])
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 1 seconds


In [48]:
df_eng['preprocessed_text'] = processedtext

C:\Users\dayal\AppData\Local\Temp\ipykernel_3032\555173176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['preprocessed_text'] = processedtext


In [49]:
df_eng.head(30)

,Unnamed: 0,user_id,username,id,date,Tweet,lang,replyCount,followersCount,preprocessed_text
0,0,1564135521760096258,SaffronFire,1570051004053491713,2022-09-14 14:05:04+00:00,Sadhus getting keeled In the name of Fake Bacc...,en,0,597,sadhu getting keeled in the name of fake bacch...
1,1,306357166,SimplyyJKS,1570040889472618497,2022-09-14 13:24:52+00:00,Need this much of confidence 😆 \n\n#LoveJihad ...,en,0,32,need this much of confidence URL
5,5,1393917252303417348,soldierofgod108,1570031351554002946,2022-09-14 12:46:58+00:00,India now chants Bharat Mata ki Jai / Vande Ma...,en,0,76,india now chant bharat mata ki jai vande matra...
6,6,1535688169231831040,RanaJi19861986,1570028420788293640,2022-09-14 12:35:19+00:00,@HinduBoy09 Tight slap for #SarTanSeJuda sloga...,en,0,0,tight slap for slogan and supporting gang
7,7,1566728648484331524,AaliyaFatima94,1570025110635008002,2022-09-14 12:22:10+00:00,What is going on?? avoid such kinda of hate......,en,1,20,what is going on avoid such kinda of hate or t...
8,8,80183148,skgrb,1570022935674638337,2022-09-14 12:13:32+00:00,@swati_gs they need to move to 52 islamic nat...,en,0,196,they need to move to 52 islamic nation special...
9,9,76088793,drrmittal,1570021154609582080,2022-09-14 12:06:27+00:00,"@SpiritOfCongres, Translation of below Tweet :...",en,0,669,translation of below tweet english version of ...
10,10,1482025421080514569,abhishGa,1570017739317850121,2022-09-14 11:52:53+00:00,#Bengal senior BJP party members are beaten up...,en,3,6635,senior bjp party member are beaten up by polic...
11,11,1482025421080514569,abhishGa,1570012954946580483,2022-09-14 11:33:52+00:00,#Bengal the BJP party members are beaten up by...,en,0,6635,the bjp party member are beaten up by police a...
14,14,1493556050557308930,Vijayafourth,1570010125859188737,2022-09-14 11:22:38+00:00,@DefenseCounselV @LiveLawIndia Who is communal...,en,0,392,who is communal community which flouts the rul...


In [50]:
df_eng.to_csv(output_csv, index = False, header=True)